In [1]:
# Librerías estándar
import os
import gzip
import pickle
import zipfile
import json

# Manipulación y análisis de datos
import pandas as pd
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, precision_score,
    recall_score, f1_score, confusion_matrix
)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, chi2
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

### **Paso 1: Realice la limpieza de los datasets:**
 - Renombre la columna "default payment next month" a "default".
 - Remueva la columna "ID".
 - Elimine los registros con informacion no disponible.
 - Para la columna EDUCATION, valores > 4 indican niveles superiores
   de educación, agrupe estos valores en la categoría "others".
 - Renombre la columna "default payment next month" a "default"
 - Remueva la columna "ID".

In [2]:
# 1.1 Cargar los datos
train_data = pd.read_csv("../files/input/train_data.csv.zip", index_col=False, compression="zip")
test_data = pd.read_csv("../files/input/test_data.csv.zip", index_col=False, compression="zip")

# 1.2 Organizar los datasets:

def limpiar(df):
    df = df.rename(columns={'default payment next month': 'default'})
    df.drop('ID', axis=1, inplace=True)
    df['EDUCATION'] = df['EDUCATION'].apply(lambda x: 4 if x>4 else x)
    df = df.query('MARRIAGE != 0 and EDUCATION != 0')
    df = df.dropna()
    return df


train_data = limpiar(train_data)
test_data = limpiar(test_data)

### **Paso 2: Divida los datasets en x_train, y_train, x_test, y_test.**

In [3]:
x_train = train_data.drop(columns=["default"])
y_train = train_data["default"]

x_test = test_data.drop(columns=["default"])
y_test = test_data["default"]

 ### **Paso 3: Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:**
 - Transforma las variables categoricas usando el método one-hot-encoding.
 - Descompone la matriz de entrada usando PCA. El PCA usa todas las componentes.
 - Estandariza la matriz de entrada.
 - Selecciona las K columnas mas relevantes de la matrix de entrada.
 - Ajusta una maquina de vectores de soporte (svm).


In [4]:
colc = ['SEX','EDUCATION','MARRIAGE']
numc= ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6' ]

# Preprocesamiento: Transformación
pre = ColumnTransformer(
transformers=[
    ("ohe",OneHotEncoder(dtype=int), colc)
],
remainder='passthrough'
)

# Pipeline

pipeline = Pipeline(steps=[
    ('transformer', pre),
    ('pca', PCA()),
    ("scaler", StandardScaler()),
    ('feature_selector', SelectKBest(f_classif)),
    ('svm',  SVC(random_state =17))
])

# Precisión: 
pipeline.fit(x_train, y_train)
print("Precisión:", pipeline.score(x_test, y_test))

Precisión: 0.8294910346363737


### **Paso 4: Optimice los hiperparametros del pipeline usando validación cruzada. Use 10 splits para la validación cruzada. Use la función de precision balanceada para medir la precisión del modelo.**

In [5]:
param_grid = {
    'feature_selector__k': [4],
    'svm__C': [13.5],
    'svm__kernel': ['rbf'],
    'svm__gamma': ['auto'],
    'svm__degree': [2],
}


grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=StratifiedKFold(10),
    scoring='balanced_accuracy',
    n_jobs=-1,
    refit=True,
    verbose=True
)

grid_search.fit(x_train, y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


c:\Users\USUARIO\OneDrive\Escritorio\2024-2-LAB-03-prediccion-del-default-usando-svc-macarvajall\.venv\lib\site-packages\sklearn\compose\_column_transformer.py:1651: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe',
                                                                         OneHotEncoder(dtype=<class 'int'>),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE'])])),
                                       ('pca', PCA()),
                                       ('scaler', StandardScaler()),
                                       ('feature_selector', SelectKBest()),
                                       ('svm', SVC(random_state=17))]),
             n_jobs=-1,
             param_grid={'feature_selector__k': [4], 'svm__C': [13.5],
                         'svm__degree': [2], 'svm__gamma': ['auto'],
                         'svm__kernel': ['rbf']},
             scoring='balanced_accuracy', verbose=True)

### **Paso 5: Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz". Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.**

In [6]:
os.makedirs('../files/models', exist_ok=True)

with gzip.open('../files/models/model.pkl.gz', 'wb') as file:
    pickle.dump(grid_search, file)

 ### **Paso 6: Calcule las metricas de precision, precision balanceada, recall, y f1-score para los conjuntos de entrenamiento y prueba. Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo. Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba.**

In [7]:
def cargar_modelo_y_predecir(data, modelo_path="../files/models/model.pkl.gz"):
    """
    Carga un modelo guardado en un archivo comprimido y realiza predicciones sobre los datos proporcionados.
    
    Args:
        data (array-like): Datos sobre los que se realizarán las predicciones.
        modelo_path (str): Ruta al archivo comprimido que contiene el modelo guardado.

    Returns:
        array-like: Predicciones realizadas por el modelo.
    """
    try:
        with gzip.open(modelo_path, "rb") as file:
            estimator = pickle.load(file)
        return estimator.predict(data)
    except FileNotFoundError:
        raise FileNotFoundError(f"No se encontró el archivo de modelo en la ruta especificada: {modelo_path}")
    except Exception as e:
        raise RuntimeError(f"Error al cargar el modelo o realizar predicciones: {e}")

# Uso de la función
y_train_pred = cargar_modelo_y_predecir(x_train)
y_test_pred = cargar_modelo_y_predecir(x_test)

In [8]:
def escribir_metricas(dict_metricas):
    models_dir = '../files/output'
    os.makedirs(models_dir, exist_ok=True)
    
    if os.path.exists('../files/output/metrics.json'):
        with open('../files/output/metrics.json', mode='r') as file:
            if len(file.readlines()) >= 4:
                os.remove('../files/output/metrics.json')
    
    with open('../files/output/metrics.json', mode='a') as file:
        file.write(str(dict_metricas).replace("'",'"')+"\n")

        
def eval_metrics(dataset,y_true, y_pred):
    accuracy = float(accuracy_score(y_true, y_pred))
    precision= float(precision_score(y_true, y_pred))
    balanced_accuracy = float(balanced_accuracy_score(y_true, y_pred))
    recall = float(recall_score(y_true, y_pred))
    f1 = float(f1_score(y_true, y_pred))
    metrics= {
        "type": "metrics",
        "dataset": dataset,
        "precision": precision,
        "balanced_accuracy": balanced_accuracy,
        "recall": recall,
        "f1_score": f1
    }
    
    escribir_metricas(metrics)

metrics_train = eval_metrics('train',y_train,y_train_pred)
metrics_test = eval_metrics('test',y_test,y_test_pred)

### **Paso 7: Calcule las matrices de confusion para los conjuntos de entrenamiento y prueba. Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo de entrenamiento o prueba.**

In [9]:
def matriz_confusion(dataset,y_true, y_pred):
    matriz = confusion_matrix(y_true, y_pred)
    matrix_confusion = {
        "type":"cm_matrix",
        "dataset": dataset,
        "true_0":{
            "predicted_0": int(matriz[0,0]),
            "predicted_1": None
        },
        "true_1":{
            "predicted_0": None,
            "predicted_1": int(matriz[1,1])
        }  
    }
    
    escribir_metricas(json.dumps(matrix_confusion))


metrics_train = matriz_confusion('train',y_train,y_train_pred)
metrics_test = matriz_confusion('test',y_test,y_test_pred)